In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
df = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
sns.heatmap(df.corr())
# We can not find out about the most important factors because the target is binary

In [ ]:
# for the sake of data exploration we can create a model to predict the most
#  important factors
x = df.iloc[:,:-1]
# getting the features alone
y = df.iloc[:,-1]
# the target
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
model = ExtraTreesClassifier()
model2 = StackingClassifier
model.fit(x,y)
feat_importance = pd.Series(model.feature_importances_, index=x.columns)
feat_importance.nlargest(12).plot(kind='barh')
# plotting using pandas plot command
# .nlargest() easy way to sort
# the plot's kind could be 'bar' or 'barh' for horizontal bars

In [ ]:
x = df.iloc[:,:-1]
# getting the features alone
y = df.iloc[:,-1]
# the target
from sklearn.ensemble import AdaBoostClassifier
model2 = AdaBoostClassifier(learning_rate=0.1)
model2.fit(x,y)
feat_importance = pd.Series(model2.feature_importances_, index=x.columns)
feat_importance.nlargest(12).plot(kind='barh')

In [ ]:
# The largest three features in affect on the death are the most important in
# finding the outliers.

In [ ]:
# we should find outliers
# Boxplots are good for finding outliers

In [ ]:
sns.boxplot(df['ejection_fraction']);
# there are two outliers

In [ ]:
df[df['ejection_fraction']>=70]

In [ ]:
df = df[df['ejection_fraction']<70]
# Removed two outliers from one of the three most important features

In [ ]:
sns.boxplot(x=df['time'],color='red')
# no outliers

In [ ]:
sns.boxplot(x=df['serum_creatinine'])

In [ ]:
sns.boxplot(y=df['age'],x=df['DEATH_EVENT'])

In [ ]:
sns.boxplot(y=df['time'],x=df['DEATH_EVENT'])

In [ ]:
df['sex'].value_counts()

In [ ]:
sns.countplot(x=df['DEATH_EVENT'],hue=df['sex']);
sns.set()

In [ ]:
plt.pie(x=df['sex'].value_counts(),labels=['Male','Female'])

In [ ]:
# male_died = len(df[(df['DEATH_EVENT']==1)&(df['sex']==1)])
# male_survived = len(df[(df['DEATH_EVENT']==0)&(df['sex']==1)])
# female_died = len(df[(df['DEATH_EVENT']==1)&(df['sex']==0)])
# female_survived = len(df[(df['DEATH_EVENT']==0)&(df['sex']==0)])

sex_mortality = []
sex_mortality.append(len(df[(df['DEATH_EVENT']==1)&(df['sex']==1)]))
sex_mortality.append(len(df[(df['DEATH_EVENT']==0)&(df['sex']==1)]))
sex_mortality.append(len(df[(df['DEATH_EVENT']==1)&(df['sex']==0)]))
sex_mortality.append(len(df[(df['DEATH_EVENT']==0)&(df['sex']==0)]))
sex_labels = ['male_died','male_survived','female_died','female_survived']

plt.pie(x=sex_mortality,autopct='%.1f',labels=sex_labels);
# autopct = '%.1f'

In [ ]:
smoking_died = len(df[(df['DEATH_EVENT']==1)&(df['smoking']==1)])
smoking_survived = len(df[(df['DEATH_EVENT']==0)&(df['smoking']==1)])
non_smoking_died = len(df[(df['DEATH_EVENT']==1)&(df['smoking']==0)])
non_smoking_survived = len(df[(df['DEATH_EVENT']==0)&(df['smoking']==0)])

smoking_mortality = []
smoking_mortality.append(len(df[(df['DEATH_EVENT']==1)&(df['smoking']==1)]))
smoking_mortality.append(len(df[(df['DEATH_EVENT']==0)&(df['smoking']==1)]))
smoking_mortality.append(len(df[(df['DEATH_EVENT']==1)&(df['smoking']==0)]))
smoking_mortality.append(len(df[(df['DEATH_EVENT']==0)&(df['smoking']==0)]))


smoking_labels = ['smoking_died','smoking_survived','non_smoking_died','non_smoking_survived']

plt.pie(x=smoking_mortality,autopct='%.1f',labels=smoking_labels);

In [ ]:
df.columns

In [ ]:
# enough of exploration
# choosing some of the most important features for the learning
X = df.drop([ 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'high_blood_pressure', 'platelets', 'sex', 'smoking','age','serum_sodium',
       'DEATH_EVENT'],axis=1)
y = df['DEATH_EVENT']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

1. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_ac = accuracy_score(y_test,lr_pred)
lr_con = confusion_matrix(y_test, lr_pred)
accuracies = []
accuracies.append(lr_ac)
print(lr_ac)
print(lr_con)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
knn = KNeighborsClassifier()
param_grid_knn = {'n_neighbors':list(range(3,9)), 'metric':['minkowski']}
knn_grid = GridSearchCV(estimator=knn,param_grid=param_grid_knn,cv=70,scoring='accuracy')
knn_grid.fit(X_train,y_train)
knn_pred = knn_grid.predict(X_test)

knn_ac = accuracy_score(y_test,knn_pred)
knn_con = confusion_matrix(y_test, knn_pred)
accuracies.append(knn_ac)
print(knn_ac)
print(knn_con)

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC(  )
param_grid_svc = {'C':[0.1,0.4,0.5,0.6,1],'degree':[1,2,3],'max_iter':[10000]}
svc_grid = GridSearchCV(estimator=svc,param_grid=param_grid_svc,cv=70,scoring='accuracy')
svc_grid.fit(X_train,y_train)
svc_pred = svc_grid.predict(X_test)

svc_ac = accuracy_score(y_test,svc_pred)
svc_con = confusion_matrix(y_test, svc_pred)
accuracies.append(svc_ac)
print(svc_ac)
print(svc_con)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rndf = RandomForestClassifier(n_estimators = 10, criterion='entropy', random_state=0)
rndf.fit(X_train,y_train)
rndf_pred = rndf.predict(X_test)

rndf_ac = accuracy_score(y_test,rndf_pred)
rndf_con = confusion_matrix(y_test, rndf_pred)
accuracies.append(rndf_ac)
print(rndf_ac)
print(rndf_con)

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
param_grid_xgb = {'n_estimators' : list(range(8,12)), 'max_depth':list(range(10,15)), 'subsample':[0.2,0.7]}
xgb_grid = GridSearchCV(estimator=xgb,param_grid=param_grid_xgb,cv=70)

xgb_grid.fit(X_train,y_train)
xgb_pred = svc_grid.predict(X_test)

xgb_ac = accuracy_score(y_test,xgb_pred)
xgb_con = confusion_matrix(y_test, xgb_pred)
accuracies.append(xgb_ac)
print(xgb_ac)
print(xgb_con)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(X_train,y_train)
gb_pred = gb.predict(X_test)
gb_ac = accuracy_score(y_test,gb_pred)
gb_con = confusion_matrix(y_test, gb_pred)
accuracies.append(gb_ac)
print(gb_ac)
print(gb_con)


In [ ]:
models = ["Logistic Regression", "KNearestNeighbours","SupportVectorClassifier","RandomForest", "XGBOOST", 'GradientBoosting']
ax = sns.barplot(x=accuracies,y=models)
